# Loan Repayment 
## Exploratory Data Analysis (EDA)

In [14]:
import os
# Change directory to this file's directory
this_path = globals()['_dh'][-1]
os.chdir(this_path)
print("This file's directory:", os.getcwd())
# Change current path to parent of this file's directory
# to access all modules from parent
os.chdir('..')
source_path = os.getcwd()
print("Parent directory:", source_path)

This file's directory: /home/mattkhoo/Git-Loan-Repayment-EDA-Predictive/notebooks
Parent directory: /home/mattkhoo/Git-Loan-Repayment-EDA-Predictive


In [2]:
from main.data_loader import DataLoader
import main.viz_utils as viz
%matplotlib inline

## Load Data

In [26]:
# Load feather data
os.chdir(source_path)
feather_path = os.path.join(str(os.getcwd()),'data-feathers') 
for _, _, files in os.walk(feather_path):
    for file in files: print(file)

clarity_underwriting_variables
payment
loan


In [27]:
# Load loan data
loan_feather_path = os.path.join(feather_path, 'loan')
loan_manager = DataLoader()
loan_manager.load_feather(loan_feather_path)
loan_manager.display(10, True)  # display first n rows

,loanId,anon_ssn,payFrequency,apr,applicationDate,originated,originatedDate,nPaidOff,approved,isFunded,loanStatus,loanAmount,originallyScheduledPaymentAmount,state,leadType,leadCost,fpStatus,clarityFraudId,hasCF
0,LL-I-07399092,beff4989be82aab4a5b47679216942fd,B,360.00,2016-02-23T17:29:01.940000,False,None,0.0,False,0,Withdrawn Application,500.0,978.27,IL,bvMandatory,6,None,5669ef78e4b0c9d3936440e6,1
1,LL-I-06644937,464f5d9ae4fa09ece4048d949191865c,B,199.00,2016-01-19T22:07:36.778000,True,2016-01-20T15:49:18.846000,0.0,True,1,Paid Off Loan,3000.0,6395.19,CA,prescreen,0,Checked,569eb3a3e4b096699f685d64,1
2,LL-I-10707532,3c174ae9e2505a5f9ddbff9843281845,B,590.00,2016-08-01T13:51:14.709000,False,None,0.0,False,0,Withdrawn Application,400.0,1199.45,MO,bvMandatory,3,None,579eab11e4b0d0502870ef2f,1
3,LL-I-02272596,9be6f443bb97db7e95fa0c281d34da91,B,360.00,2015-08-06T23:58:08.880000,False,None,0.0,False,0,Withdrawn Application,500.0,1074.05,IL,bvMandatory,3,None,555b1e95e4b0f6f11b267c18,1
4,LL-I-09542882,63b5494f60b5c19c827c7b068443752c,B,590.00,2016-06-05T22:31:34.304000,False,None,0.0,False,0,Rejected,350.0,814.37,NV,bvMandatory,3,None,5754a91be4b0c6a2bf424772,1
5,LL-I-09734486,b5541f49472fa0fce8e473306768f7fb,M,650.00,2016-06-12T19:27:47.951000,False,None,0.0,False,0,Withdrawn Application,300.0,738.18,IN,organic,0,None,574e2029e4b061d2c3a16a4c,1
6,LL-I-15006968,1828c64bb2dffeae88b27174a9f79ecc,B,680.00,2017-01-12T18:04:37.921000,False,None,0.0,False,0,Withdrawn Application,400.0,1362.92,TX,lead,25,None,5877c533e4b08f2480ab7e5a,1
7,LL-I-08327946,02596517e7633c7e87e6b333a0fb1bbe,M,449.99,2016-04-01T22:13:02.131000,False,None,0.0,False,0,Withdrawn Application,350.0,759.84,UT,bvMandatory,3,None,None,0
8,LL-I-10568307,523ed92e1145eb688bb631da24695197,S,251.00,2016-07-27T00:19:52.808000,False,None,0.0,False,0,Withdrawn Application,2600.0,8230.01,CA,california,40,None,5797fe22e4b0d05020f298ce,1
9,LL-I-05518977,47bf79119075e41ef65510f2900c8e4a,B,360.00,2015-12-09T18:17:33.622000,False,None,0.0,False,0,Withdrawn Application,500.0,995.22,IL,bvMandatory,6,None,563281d2e4b07887adc5bc0a,1


In [28]:
# Load payment data
payment_feather_path = os.path.join(feather_path, 'payment')
payment_manager = DataLoader()
payment_manager.load_feather(payment_feather_path)
payment_manager.display(10, False) # display last n rows

,loanId,installmentIndex,isCollection,paymentDate,principal,fees,paymentAmount,paymentStatus,paymentReturnCode
689354,LL-I-18629478,3,False,2017-04-28T04:00:00,33.39,29.90,63.29,None,None
689355,LL-I-18629478,4,False,2017-05-15T04:00:00,29.78,33.51,63.29,None,None
689356,LL-I-18629478,5,False,2017-05-31T04:00:00,34.11,29.18,63.29,None,None
689357,LL-I-18629478,6,False,2017-06-15T04:00:00,38.46,24.83,63.29,None,None
689358,LL-I-18629478,7,False,2017-06-30T04:00:00,41.30,21.99,63.29,None,None
689359,LL-I-18629478,8,False,2017-07-14T04:00:00,45.62,17.67,63.29,None,None
689360,LL-I-18629478,9,False,2017-07-31T04:00:00,45.67,17.62,63.29,None,None
689361,LL-I-18629478,10,False,2017-08-15T04:00:00,51.12,12.17,63.29,None,None
689362,LL-I-18629478,11,False,2017-08-31T04:00:00,54.35,8.94,63.29,None,None
689363,LL-I-18629478,12,False,2017-09-15T04:00:00,58.83,4.36,63.19,None,None


In [29]:
# Load underwriting data
undw_feather_path = os.path.join(feather_path, 'clarity_underwriting_variables')
undw_manager = DataLoader()
undw_manager.load_feather(undw_feather_path)
undw_manager.display(10, False) # display last n rows

,.underwritingdataclarity.clearfraud.clearfraudinquiry.thirtydaysago,.underwritingdataclarity.clearfraud.clearfraudinquiry.twentyfourhoursago,.underwritingdataclarity.clearfraud.clearfraudinquiry.oneminuteago,.underwritingdataclarity.clearfraud.clearfraudinquiry.onehourago,.underwritingdataclarity.clearfraud.clearfraudinquiry.ninetydaysago,.underwritingdataclarity.clearfraud.clearfraudinquiry.sevendaysago,.underwritingdataclarity.clearfraud.clearfraudinquiry.tenminutesago,.underwritingdataclarity.clearfraud.clearfraudinquiry.fifteendaysago,.underwritingdataclarity.clearfraud.clearfraudinquiry.threesixtyfivedaysago,.underwritingdataclarity.clearfraud.clearfraudindicator.inquiryonfilecurrentaddressconflict,...,.underwritingdataclarity.clearfraud.clearfraudidentityverification.phonematchtypedescription,.underwritingdataclarity.clearfraud.clearfraudidentityverification.overallmatchresult,.underwritingdataclarity.clearfraud.clearfraudidentityverification.phonetype,.underwritingdataclarity.clearfraud.clearfraudidentityverification.ssndobreasoncode,.underwritingdataclarity.clearfraud.clearfraudidentityverification.ssnnamereasoncode,.underwritingdataclarity.clearfraud.clearfraudidentityverification.nameaddressreasoncode,.underwritingdataclarity.clearfraud.clearfraudidentityverification.ssndobmatch,.underwritingdataclarity.clearfraud.clearfraudidentityverification.overallmatchreasoncode,clearfraudscore,underwritingid
49742,5.0,3.0,3.0,3.0,5.0,3.0,3.0,5.0,48.0,False,...,(M) Mobile Phone,partial,None,None,None,None,match,11.0,567.0,58cc510ce4b0b0437e1c4df5
49743,3.0,3.0,3.0,3.0,6.0,3.0,3.0,3.0,7.0,False,...,(M) Mobile Phone,match,None,None,None,None,match,1.0,771.0,58cc51b6e4b0b0437e1ddc64
49744,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,9.0,False,...,(M) Mobile Phone,partial,None,None,None,None,match,11.0,585.0,58cc52c7e4b0b0437e1ee2b1
49745,11.0,11.0,1.0,11.0,11.0,11.0,11.0,11.0,14.0,False,...,(M) Mobile Phone,partial,None,None,None,None,match,11.0,582.0,58cc5577e4b0b0437e23cbcb
49746,3.0,3.0,1.0,3.0,8.0,3.0,3.0,3.0,15.0,False,...,(M) Mobile Phone,match,None,None,None,None,match,1.0,834.0,58cc56d4e4b0b0437e26556b
49747,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,24.0,False,...,(M) Mobile Phone,partial,None,None,None,None,match,11.0,677.0,58cc5706e4b0b0437e26b19c
49748,6.0,4.0,1.0,4.0,11.0,4.0,1.0,4.0,17.0,False,...,(M) Mobile Phone,partial,None,None,None,None,match,16.0,640.0,58cc5763e4b0b0437e272d38
49749,4.0,4.0,1.0,4.0,4.0,4.0,1.0,4.0,5.0,False,...,(M) Mobile Phone,match,None,None,None,None,match,1.0,576.0,58cc57d4e4b0b0437e278bff
49750,3.0,3.0,2.0,2.0,3.0,3.0,2.0,3.0,25.0,False,...,(M) Mobile Phone,match,None,None,None,None,match,1.0,830.0,58cc5916e4b0b0437e2b6e07
49751,5.0,3.0,2.0,2.0,6.0,3.0,2.0,4.0,12.0,False,...,(M) Mobile Phone,partial,None,None,None,A8,match,6.0,793.0,58cc660ce4b0b0437e47de4a
